<a href="https://colab.research.google.com/github/yandexdataschool/MLatImperial2022/blob/main/Seminars/lab_01_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Metric methods: KNN

In [ ]:
!wget -N https://raw.githubusercontent.com/yandexdataschool/MLatImperial2022/main/Seminars/utility.py

In [ ]:
import sklearn as skl
import pandas as pd
import numpy as np
from sklearn.neighbors  import KNeighborsClassifier
import numpy as np
from  utility import plot_predictions_2D
from sklearn.model_selection import train_test_split
from sklearn import datasets
import matplotlib.pyplot as plt
import matplotlib.cm as colors
random_state=0

###  How does KNN work on toy oval noisy classes ?

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
random_state=42
num_samples = 300
X, y = datasets.make_circles(n_samples=num_samples, factor=0.5, noise=.3,random_state=random_state)

X[:,1]=X[:,1]/5
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)
plt.scatter(X_train[:,0],X_train[:,1],c=y_train,cmap='bwr')

In [ ]:
for n in [2,10,20]:
    knn_model=KNeighborsClassifier(n_neighbors=n)
    knn_model.fit(X_train,y_train)
    Y_predicted = knn_model.predict(X_test)
    accuracy = accuracy_score(Y_predicted,y_test)
    
    plot_predictions_2D(
        knn_model, X_train, y_train,train=False, task='classification', n=100, cmap=colors.ocean, feature_names=['x1','x2'],alpha=0.5)
    plt.title('ovals K=%d test accuracy=%.3f'%(n,accuracy))   

### Featues scaling


To scale features, we will use the transformer [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html).

Transformers in `sklearn` have methods `fit` and `transform` (and also `fit_transform`). The `fit` method takes a training sample as input and calculates the necessary values ​​from it (for example, statistics like `StandardScaler`: the mean and standard deviation of each feature). `transform` applies a transformation to the given dataset.

The most common scalers:
- **StandardScaler()**
- **MinMaxScaler()**

other scalers see in sklearn.preprocessing



In [ ]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()

X_train_norm = scaler.fit_transform(X_train)
X_test_norm=scaler.transform(X_test)
for n in [2,20,35]:
    knn_model=KNeighborsClassifier(n_neighbors=n)
    knn_model.fit(X_train_norm,y_train)
    Y_predicted = knn_model.predict(X_test_norm)
    accuracy = accuracy_score(Y_predicted,y_test)
    plot_predictions_2D(
        knn_model, X_train_norm, y_train, task='classification', n=100, cmap=colors.ocean, feature_names=['x1','x2'],alpha=0.5)
    plt.title('scaled ovals K=%d test accuracy=%.3f'%(n,accuracy))

 Feature scaling improved accuracy


In [ ]:
#<YOUR TURN>
# change scaler in previous cell to another  (MinMaxScaler, RobustScaler or other)
# does it give improvement in accuracy_score?

## Parameteres search

In [ ]:
from utility import show_param_dependency
knn = KNeighborsClassifier(p=2)
show_param_dependency(knn, X_train_norm, y_train, X_test_norm, y_test, param_name='n_neighbors', 
     
                      loss_fun='error_rate', param_vals=np.arange(1,100,1), x_label='K')

In [ ]:
knn = KNeighborsClassifier(n_neighbors=35)
show_param_dependency(knn, X_train_norm, y_train, X_test_norm, y_test, param_name='p', 
     
                      loss_fun='error_rate', param_vals=np.arange(1,10,1), x_label='K')

In [ ]:
#<YOUR TURN>
#try to find best combination of p and n_neighbors using these graphs
#fit the model with these parameters and make another train-test split, test your model on the new test dataset. did you get similar accuracy?
#QUESTION
#is it really the best combination of p and n_neighbors?

### Cross validation score

<img src="https://docs.splunk.com/images/thumb/e/ee/Kfold_cv_diagram.png/1200px-Kfold_cv_diagram.png" width=50%>


In cross-validation, we divide the training set into $n$ parts (folds). Then we train $n$ models: each model is trained in the absence of a corresponding fold, that is, the $i$-th model is trained on the entire training set, except for objects that are in the $i$-th fold (out-of-fold). After that we measure the quality of the $i$th model on the $i$th fold. Since it did not participate in the training of this model, we will get an "honest result". After that, to get the final value of the accuracy (or other) metric, we can average the $n$ values we got.


https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html

In [ ]:
#Question: what's the best number of folds?

In [ ]:
from sklearn.model_selection import cross_val_score
knn_classifier = KNeighborsClassifier(n_neighbors=35,p=2)
cv_scores = cross_val_score(knn_classifier, X_train_norm, y_train, cv=5, scoring="accuracy")
print("Cross-validation scores : ", cv_scores)
print(f"Cross-validation scores mean:",cv_scores.mean())
    

In [ ]:
#<YOUR TURN>
#test some n_neighbors (for example, from the list [1,3,8,15,20,35] ) using 'for' cycle on cross-validation
#print cross-validation scores for all tested n_neighbors

##  GridSearchCV
<img src="https://quantmetry.b-cdn.net/wp-content/uploads/2019/11/corpsTME1.png" width="100%" />

<img src="https://www.oreilly.com/library/view/statistics-for-machine/9781788295758/assets/ac3f2f5a-9199-4bb7-8ce6-47e4dc307a0e.png" width="60%" />


*class sklearn.model_selection.GridSearchCV(estimator, param_grid, , scoring=None, n_jobs=None, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)*

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

Grid Search:
In Grid Search, the possible values of hyperparameters are defined into the set. Then these sets of possible values of hyperparameters are combined by using Cartesian product and form a multidimensional grid. 

The grid search then examines all combinations of these hyperparameters

For each possible combination of hyperparameters, we train a model on them

The hyperparameters associated with the highest accuracy are then returned

A grid search is guaranteed to examine all possible combinations of hyperparameters. The problem is that the more hyperparameters you have, the more the number of combinations grows exponentially.
If the hyperparameter search space is large, it takes a lot of time and computational power to optimize the hyperparameter.
There is no guarantee that these algorithms find local maximum.

Grid Search is an exhaustive search method

In [ ]:
from sklearn.model_selection import GridSearchCV
knn = KNeighborsClassifier()
param_grid = {'n_neighbors':[1,3,5,8,10,16,15,20,35,40], 'p':[1,2,3,4,6]}
grid_knn = GridSearchCV(knn, param_grid, n_jobs=-1, refit=True)
grid_knn.fit(X_train_norm,y_train)
print(grid_knn.best_score_, grid_knn.best_params_)

In [ ]:
print("Test accuracy:", accuracy_score(y_test, grid_knn.best_estimator_.predict(X_test_norm)))

Look at default parametrs of kNN.

In [ ]:
knn_classifier = KNeighborsClassifier()
knn_classifier.get_params()

f=knn_classifier.set_params(**grid_knn.best_params_)
knn_classifier.get_params()

## Iris species task

https://en.wikipedia.org/wiki/Iris_flower_data_set

The data set consists of 50 samples from each of three species of Iris (Iris setosa, Iris virginica and Iris versicolor). Four features were measured from each sample: the length and the width of the sepals and petals, in centimeters.

### Spliting dataset into training set and test set

In [ ]:
iris = datasets.load_iris()
X = iris.data
y = iris.target
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

### 3D visualization

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(1, figsize=(9, 6))
ax = Axes3D(fig, elev=48, azim=134)
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=y,
           cmap=plt.cm.Set1, edgecolor='k', s = X[:, 3]*50)

for name, label in [('Virginica', 0), ('Setosa', 1), ('Versicolour', 2)]:
    ax.text3D(X[y == label, 0].mean(),
              X[y == label, 1].mean(),
              X[y == label, 2].mean(), name,
              horizontalalignment='center',
              bbox=dict(alpha=.5, edgecolor='w', facecolor='w'),size=25)

ax.set_title("3D visualization", fontsize=40)
ax.set_xlabel("Sepal Length [cm]", fontsize=25)
ax.w_xaxis.set_ticklabels([])
ax.set_ylabel("Sepal Width [cm]", fontsize=25)
ax.w_yaxis.set_ticklabels([])
ax.set_zlabel("Petal Length [cm]", fontsize=25)
ax.w_zaxis.set_ticklabels([])

plt.show()

In [ ]:

knn = KNeighborsClassifier()
param_grid = {'n_neighbors':[1,3,5,8,10,15,20], 'p':[1,2,4,6]}
grid_knn = GridSearchCV(knn, param_grid, n_jobs=-1, refit=True)
grid_knn.fit(X_train,y_train)
print(grid_knn.best_score_, grid_knn.best_params_)

y_predicted = grid_knn.predict(X_test)
accuracy = accuracy_score(y_test,y_predicted)
print(f"accuracy={%.3f}", accuracy)

## Pipeline

Now we will use a very convenient class [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): model training is often represented as a sequence of some actions with training and test sets ( for example, first you need to scale the sample (and for the training sample you need to apply the `fit` method, and for the test sample &mdash; `transform`), and then train / apply the model (for the training `fit`, and for the test &mdash; `predict`) `Pipeline` allows you to store this sequence of steps and correctly handles different types of samples: both training and test.

 [`Pipeline`](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html):

 sklearn.pipeline.**Pipeline**(steps, *, memory=None, verbose=False)
 
 sklearn.pipeline.**make_pipeline**(*steps, memory=None, verbose=False)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

 # define the pipeline object.
pipeline = Pipeline(steps = [
              ('scaler', MinMaxScaler())
 
              ,('KNN',KNeighborsClassifier())
           ])

parameteres = {'KNN__n_neighbors':[1,2,5,10], 'KNN__p':[1,2,5]}
grid = GridSearchCV(pipeline, param_grid=parameteres, cv=5)
grid.fit(X_train, y_train)

y_predicted = grid_knn.predict(X_test)
accuracy = accuracy_score(y_test,y_predicted)
print("accuracy=%.3f" %accuracy)


In [ ]:
#<YOUR TURN>
# try to change scaler. is there any improvements?
# instead of parameter p  try changing 'metric' parameter

# Nadaraya-Watson kernel regression






In [ ]:
### Ridge regression
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
#https://scikit-learn.org/stable/modules/generated/sklearn.kernel_ridge.KernelRidge.html

rng = np.random.RandomState(0)

X = 15 * rng.rand(100, 1)
y = np.sin(X).ravel()
y += 3 * (0.5 - rng.rand(X.shape[0])) 

import warnings
warnings.filterwarnings("ignore")

from sklearn.kernel_ridge import KernelRidge

rng = np.random.RandomState(0)

X = 15 * rng.rand(100, 1)
y = np.sin(X).ravel()
y += 3 * (0.5 - rng.rand(X.shape[0]))

#-----------------------

kr_linear = KernelRidge()
kr_linear.fit(X, y)

#-----------------------

kr_rbf = KernelRidge(kernel="rbf",gamma=0.2)

kr_rbf.fit(X, y)

#-----------------------

param_grid = {"gamma": [1e1,1e0, 1e-1, 1e-2, 1e-3],
              "kernel":  ["rbf"]}

kr_grid = GridSearchCV(KernelRidge(), param_grid=param_grid, scoring="neg_mean_squared_error")
kr_grid.fit(X,y)

#-----------------------

y_kr_linear = kr_linear.predict(X_plot)
y_kr_rbf = kr_rbf.predict(X_plot)
y_kr_grid = kr_grid.predict(X_plot)

#-----------------------
X_plot = np.linspace(0, 20, 10000)[:, None]


plt.figure(figsize=(20, 10))
plt.scatter(X, y, c='k', label='data')
plt.plot(X_plot, np.sin(X_plot), color='blue', lw=2, label='True')
plt.plot(X_plot, y_kr_rbf, color='brown', lw=2, label='Test kernel')

plt.plot(X_plot, y_kr_grid, lw=2, label='(%s)' % kr_grid.best_params_)

plt.xlabel('data')
plt.ylabel('target')
plt.xlim(0, 20)
plt.ylim(-4, 4)

plt.title('Different Kernels')
plt.legend(loc="best",  scatterpoints=1, prop={'size': 15})
plt.show()



In [ ]:
#<YOUR TURN>
#Adjust parameter gamma to make an overfitted model. which gamma values corresponds to underfitted model? Why?
# Try polonomial kernel.Is it better than RBF-kernel?
